In [2]:
!pip install tensorflow_addons
from segnet import SegNet
from sequence import FoodSequence
from ensemble_net import EnsembleNet
import tensorflow as tf
import json
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import numpy as np

     |████████████████████████████████| 686kB 3.9MB/s 


In [3]:
def UNet(n_categories, 
         input_shape=(256, 256, 3),
         kernel=3,
         pool_size=(2,2), 
         output_mode='sigmoid'):


        inputs = Input(shape=input_shape)

        conv_1 = Convolution2D(64, (kernel, kernel), padding="same")(inputs)
        batch_1 = BatchNormalization()(conv_1)
        activation_1 = Activation("relu")(batch_1)
        conv_2 = Convolution2D(64, (kernel, kernel), padding="same")(activation_1)
        batch_2 = BatchNormalization()(conv_2)
        activation_2 = Activation("relu")(batch_2)

        pool_1 = MaxPooling2D(pool_size)(activation_2)

        conv_3 = Convolution2D(128, (kernel, kernel), padding="same")(pool_1)
        batch_3 = BatchNormalization()(conv_3)
        activation_3 = Activation("relu")(batch_3)
        conv_4 = Convolution2D(128, (kernel, kernel), padding="same")(activation_3)
        batch_4 = BatchNormalization()(conv_4)
        activation_4 = Activation("relu")(batch_4)

        pool_2 = MaxPooling2D(pool_size)(activation_4)

        conv_5 = Convolution2D(256, (kernel, kernel), padding="same")(pool_2)
        batch_5 = BatchNormalization()(conv_5)
        activation_5 = Activation("relu")(batch_5)
        conv_6 = Convolution2D(256, (kernel, kernel), padding="same")(activation_5)
        batch_6 = BatchNormalization()(conv_6)
        activation_6 = Activation("relu")(batch_6)

       
        pool_3 = MaxPooling2D(pool_size)(activation_6)

        conv_7 = Convolution2D(512, (kernel, kernel), padding="same")(pool_3)
        batch_7 = BatchNormalization()(conv_7)
        activation_7 = Activation("relu")(batch_7)
        conv_8 = Convolution2D(512, (kernel, kernel), padding="same")(activation_7)
        batch_8 = BatchNormalization()(conv_8)
        activation_8 = Activation("relu")(batch_8)

        pool_4 = MaxPooling2D(pool_size)(activation_8)

        conv_9 = Convolution2D(1024, (kernel, kernel), padding="same")(pool_4)
        batch_9 = BatchNormalization()(conv_9)
        activation_9 = Activation("relu")(batch_9)
        conv_10 = Convolution2D(1024, (kernel, kernel), padding="same")(activation_9)
        batch_10 = BatchNormalization()(conv_10)
        activation_10 = Activation("relu")(batch_10)

        # decoder

        unpool_1 = UpSampling2D(pool_size)(activation_10)

        conv_11 = Convolution2D(512, pool_size, padding="same")(unpool_1)
        batch_11 = BatchNormalization()(conv_11)
        #activation_11 = Activation("relu")(batch_11)
        merge_11 = Concatenate(axis=-1)([activation_8, batch_11])
        conv_12 = Convolution2D(512, (kernel, kernel), padding="same")(merge_11)
        batch_12 = BatchNormalization()(conv_12)
        activation_12 = Activation("relu")(batch_12)
        conv_13 = Convolution2D(512, (kernel, kernel), padding="same")(activation_12)
        batch_13 = BatchNormalization()(conv_13)
        activation_13 = Activation("relu")(batch_13)

        unpool_2 = UpSampling2D(pool_size)(activation_13)

        conv_14 = Convolution2D(256, pool_size, padding="same")(unpool_2)
        batch_14 = BatchNormalization()(conv_14)
        #activation_14 = Activation("relu")(batch_14)
        merge_14 = Concatenate(axis=-1)([activation_6, batch_14])
        conv_15 = Convolution2D(256, (kernel, kernel), padding="same")(merge_14)
        batch_15 = BatchNormalization()(conv_15)
        activation_15 = Activation("relu")(batch_15)
        conv_16 = Convolution2D(256, (kernel, kernel), padding="same")(activation_15)
        batch_16 = BatchNormalization()(conv_16)
        activation_16 = Activation("relu")(batch_16)

        unpool_3 = UpSampling2D(pool_size)(activation_16)

        conv_17 = Convolution2D(128, pool_size, padding="same")(unpool_3)
        batch_17 = BatchNormalization()(conv_17)
        #activation_17 = Activation("relu")(batch_17)
        merge_17 = Concatenate(axis=-1)([activation_4, batch_17])
        conv_18 = Convolution2D(128, (kernel, kernel), padding="same")(merge_17)
        batch_18 = BatchNormalization()(conv_18)
        activation_18 = Activation("relu")(batch_18)
        conv_19 = Convolution2D(128, (kernel, kernel), padding="same")(activation_18)
        batch_19 = BatchNormalization()(conv_19)
        activation_19 = Activation("relu")(batch_19)

        unpool_4 = UpSampling2D(pool_size)(activation_19)

        conv_20 = Convolution2D(64, pool_size, padding="same")(unpool_4)
        batch_20 = BatchNormalization()(conv_20)
        #activation_20 = Activation("relu")
        merge_20 = Concatenate(axis=-1)([activation_2, batch_20])
        conv_21 = Convolution2D(64, (kernel, kernel), padding="same")(merge_20)
        batch_21 = BatchNormalization()(conv_21)
        activation_21 = Activation("relu")(batch_21)
        conv_22 = Convolution2D(64, (kernel, kernel), padding="same")(activation_21)
        batch_22 = BatchNormalization()(conv_22)
        activation_22 = Activation("relu")(batch_22)

        conv_23 = Convolution2D(n_categories, (1, 1), padding="valid")(activation_22)
        outputs = Activation(output_mode)(conv_23)
        
        model = Model(inputs=inputs, outputs=outputs, name='U-Net')
        

        return model

In [4]:
!pip install -U aicrowd-cli
API_KEY = "67587f4b3488144e75864045bf9db5e9" #get your api-key from https://www.aicrowd.com/participants/me

!aicrowd login --api-key $API_KEY 
!aicrowd dataset download --challenge food-recognition-challenge 3 4 5
!tar -zxvf train-v0.4.tar.gz

     |████████████████████████████████| 51kB 2.9MB/s 
     |████████████████████████████████| 61kB 6.4MB/s 
     |████████████████████████████████| 81kB 5.1MB/s 
     |████████████████████████████████| 215kB 7.1MB/s 
     |████████████████████████████████| 61kB 5.2MB/s 
     |████████████████████████████████| 174kB 7.4MB/s 
     |████████████████████████████████| 51kB 4.8MB/s 
     |████████████████████████████████| 71kB 5.6MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


Streaming output truncated to the last 5000 lines.
./train/images/016635.jpg
./train/images/084765.jpg
./train/images/030078.jpg
./train/images/095272.jpg
./train/images/074596.jpg
./train/images/068251.jpg
./train/images/021057.jpg
./train/images/098101.jpg
./train/images/094285.jpg
./train/images/007293.jpg
./train/images/099408.jpg
./train/images/019538.jpg
./train/images/049791.jpg
./train/images/021889.jpg
./train/images/075978.jpg
./train/images/043050.jpg
./train/images/077428.jpg
./train/images/026325.jpg
./train/images/008192.jpg
./train/images/082412.jpg
./train/images/013239.jpg
./train/images/028513.jpg
./train/images/084736.jpg
./train/images/076279.jpg
./train/images/103474.jpg
./train/images/070743.jpg
./train/images/071644.jpg
./train/images/095290.jpg
./train/images/037048.jpg
./train/images/009347.jpg
./train/images/078783.jpg
./train/images/084642.jpg
./train/images/032499.jpg
./train/images/099724.jpg
./train/images/079743.jpg
./train/images/074193.jpg
./train/image

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
def precision(mask, y_pred, threshold=0.6):
  y = y_pred > threshold
  real_cats = tf.math.reduce_any(tf.cast(mask, dtype=tf.bool), axis=[1, 2])
  cats = tf.math.reduce_any(y, axis=[1, 2])
  true_positive = tf.math.count_nonzero(tf.boolean_mask(real_cats, cats))
  tp_plus_fp = tf.math.count_nonzero(cats)
  if tp_plus_fp == 0:
    return tf.cast(0, dtype=tf.float64);
  return true_positive / tp_plus_fp

In [7]:
def IoU(mask, y_pred, threshold=0.6):
  y = y_pred > threshold
  intersection = tf.math.count_nonzero(tf.boolean_mask(y, mask))
  union = tf.math.count_nonzero(mask + tf.cast(y, dtype=tf.float32))
  return  intersection / union

In [8]:
def precisionWithIoU(mask, y_pred, threshold_max=0.77, threshold=0.4):
  y_max = y_pred > threshold_max
  y = y_pred > threshold 
  real_cats = tf.math.reduce_any(tf.cast(mask, dtype=tf.bool), axis=[1, 2])
  cats = tf.math.reduce_any(y_max, axis=[1, 2])
  number_of_prediction = tf.math.count_nonzero(cats)
  if number_of_prediction == 0:
    return tf.cast(0, dtype=tf.float64)

  intersections = tf.math.count_nonzero(tf.logical_and(tf.cast(mask, dtype=tf.bool), y), axis=[1, 2])
  unions = tf.math.count_nonzero(tf.logical_or(tf.cast(mask, dtype=tf.bool), y), axis=[1, 2])
  ious = intersections / unions
  predicted_cats = tf.logical_and(ious > 0.5, cats)
  true_positive = tf.math.count_nonzero(tf.logical_and(real_cats, predicted_cats))

  return true_positive / number_of_prediction
  

In [9]:
def true_positive(mask, y_pred, threshold_max=0.77, threshold=0.4):
  y_max = y_pred > threshold_max
  y = y_pred > threshold 
  real_cats = tf.math.reduce_any(tf.cast(mask, dtype=tf.bool), axis=[1, 2])
  cats = tf.math.reduce_any(y_max, axis=[1, 2])

  intersections = tf.math.count_nonzero(tf.logical_and(tf.cast(mask, dtype=tf.bool), y), axis=[1, 2])
  unions = tf.math.count_nonzero(tf.logical_or(tf.cast(mask, dtype=tf.bool), y), axis=[1, 2])
  ious = intersections / unions
  predicted_cats = tf.logical_and(ious > 0.5, cats)

  return tf.math.count_nonzero(tf.logical_and(real_cats, predicted_cats))

def true_negative(mask, y_pred, threshold_max=0.77, threshold=0.4):
  y_max = y_pred > threshold_max
  real_cats = tf.math.reduce_any(tf.cast(mask, dtype=tf.bool), axis=[1, 2])
  cats = tf.math.reduce_any(y_max, axis=[1, 2])
  true_negative_chan = tf.logical_not(tf.logical_or(cats, real_cats))
  return tf.math.count_nonzero(true_negative_chan)

def false_positive(mask, y_pred, threshold_max=0.77, threshold=0.4):
  y_max = y_pred > threshold_max
  cats = tf.math.reduce_any(y_max, axis=[1, 2])
  return tf.math.count_nonzero(cats) - true_positive(mask, y_pred, threshold_max, threshold)

def false_negative(mask, y_pred, threshold_max=0.77, threshold=0.4):
  y_max = y_pred > threshold_max
  cats = tf.math.reduce_any(y_max, axis=[1, 2])
  return tf.math.count_nonzero(tf.logical_not(cats)) - true_negative(mask, y_pred, threshold_max, threshold)

In [11]:
segnet = tf.keras.models.load_model('/content/gdrive/MyDrive/challenge/models/segnet/model140', custom_objects={"SegNet": SegNet}, compile=False)

unet = tf.keras.models.load_model('/content/gdrive/MyDrive/challenge/models/unet/architecture', custom_objects={"UNet": UNet}, compile=False)
unet.load_weights('/content/gdrive/MyDrive/challenge/models/unet/weights/unet_weights_epoch100.h5')

In [12]:
model = EnsembleNet([segnet, unet])

In [13]:
file = open("train/annotations.json", "r")
annotations = json.loads(file.read())

ann = pd.DataFrame(annotations['annotations'])
mapping = {item:i for i, item in enumerate(ann["category_id"].unique())}
categories = pd.DataFrame(annotations['categories'])
categories['id'] = categories['id'].apply(lambda x: mapping[x])
categories = categories.set_index('id')

file_ = open("val/annotations.json", "r")
annotations = json.loads(file_.read())

ann = pd.DataFrame(annotations['annotations'])
ann["category_id"] = ann["category_id"].apply(lambda x: mapping[x])
annotations_map = {img_id: {} for img_id in ann['image_id'].unique()}
for index, row in ann.iterrows():
    annotations_map[row['image_id']][row['category_id']] = row['segmentation']
images = pd.DataFrame(annotations['images'])
images = images.set_index('id')

np.random.seed(0)
msk = np.random.rand(len(images)) < 0.5
val_images = images[msk]
test_images = images[~msk]

val_set = FoodSequence(val_images, annotations_map, len(categories), batch_size=1, folder='val',img_size=(256, 256), data_augmentation=False)
test_set = FoodSequence(test_images, annotations_map, len(categories), batch_size=1, folder='val',img_size=(256, 256), data_augmentation=False)

In [14]:
model.compile(metrics=[true_positive, false_positive, true_negative, false_negative])
model.evaluate(test_set)

620/620 [==============================] - 2402s 4s/step - loss: 0.0000e+00 - true_positive: 0.2935 - false_positive: 1.4952 - true_negative: 269.9290 - false_negative: 1.2823


[0.0,
 0.2935483753681183,
 1.4951612949371338,
 269.9290466308594,
 1.2822580337524414]

In [ ]:
from functools import partial

ths = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
ious = []

for threshold in ths:
  model.compile(metrics=partial(precision, threshold=threshold))
  results = model.evaluate(val_set)
  ious.append(results[1])

In [ ]:
plt.plot(ths, ious);
plt.title("Precision with different thresholds");

In [ ]:
def desired_output(img_shape, segmentations):
        desired_output = np.zeros((img_shape[0], img_shape[1], len(categories)), dtype=np.int32)

        for i in segmentations:
          t = np.zeros((img_shape[0], img_shape[1], 1), dtype=np.int32)
          for seg in segmentations[i]:
              s = np.array(seg, dtype=np.int32).reshape((-1, 2))
              t = cv2.fillPoly(t, [s], 255)

          desired_output[:, :, i] = t.squeeze()

        return desired_output / 255

In [ ]:
threshold = 0.58
img_id = val_images.sample().index.values[0]
print("Sample n°", img_id)
img = cv2.imread('val/images/' + val_images.loc[img_id, 'file_name'])

plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB));

input_img = cv2.resize(img, dsize=(256, 256)) / 255
y = model.predict(input_img.reshape((1, input_img.shape[0], input_img.shape[1], input_img.shape[2])))

img_ann = ann[ann['image_id'] == img_id]
segmentations = dict(zip(img_ann['category_id'], img_ann['segmentation']))
y_true = desired_output(img.shape, segmentations)
y_true = cv2.resize(y_true, dsize=(256, 256), interpolation=cv2.INTER_NEAREST)

print("IoU: ", IoU(y_true, y[0], threshold=threshold))

y = y > threshold
for i in range(len(categories)):
  if np.any(y[0, :, :, i]):
    fig = plt.figure()
    plt.title(categories.iloc[i]['name'] + " - max " + str(np.max(y[0, :, :, i])))
    plt.imshow(y[0, :, :, i], cmap='gray', vmin=0, vmax=1)